# Neural Network

# 1. Artifical Neural Network
- biological neural network를 모방한 computing system이다.
- layer가 두개 이상 쌓인 형태는 Deep Neural Network라고 한다.
- 간단히 생각하면 아래 그림과 같이 linear classifier를 여러번 이어놓은 형태를 의미한다. <br/>hidden node에서 input의 feature들을 우리가 지정한 hidden layer의 개수만큼으로 표현해주고 이러한 linear function을 여러번 만들어서 최종 output을 추출해내는 형태이다. <br/>
    이때 neural network가 어떤 식으로 feature들을 가공하는지 알 수 없기 때문에 blackbox algorithm이라고 하며, 기계가 사람이 알 수 없는 방식으로 필요한 feature들을 뽑아준다는 점에서 deep learning이 기존의 머신러닝 알고리즘들과 가장 큰 차이점을 보인다고 할 수 있다.<br/> 
![nn](picture/neural_network.png)
<br/>
- 그림에서 보이는 각각의 선들이 weight를 의미한다고 할 수 있다.<br/> Training과정에서 input을 넣어서 output을 계산하고 score를 계산하는 과정을 **forward propagation** 이라고 하며, loss function을 이용해서 weight를 loss를 줄이는 방향으로 update를 하는 과정을 **back propagation**이라고 한다.<br/> 이러한 과정으로 한번의 forward propagation이랑 back propagation 과정을 한번 거치는 것을 1 **epoch**라고 한다.<br/><br/>
- deep learning model은 다양한 hyperparameter에 의해서 영향을 받는다. 이러한 hyperparamter의 종류는 다음과 같다.
    - learning rate : weight를 얼마만큼 이동하면서 update할 것인지
    - number of hidden layer : layer를 얼마나 쌓을 것인지
    - number of hidden node for each layer : 각 hidden layer의 node를 몇개씩 지정해 줄것인지
    - regularization : l2또는 l1 regularization 을 얼마만큼 적용해줄 것인지
    - dropout rate : dropout시키는 비율을 얼마로 할것인지
  

# 2. Neural Network의 원리
## Forward pass

input으로 (N,D)의 X가 주어졌다고 했을 때, hidden layer가 하나인 forward pass의 계산과정은 다음과 같다. <br/>
각 layer마다 linear classifier와 동일하게 wx+b 함수를 계산하고, activation함수를 이용해서 output이 0또는 1나올 수 있게 조정해준다.<br/>
여기서는 activation function으로 ReLU함수를 이용하였다.

In [ ]:
import numpy as np

def relu(self,x):
    return np.maximum(x,0)

# parameter W와 b 불러오기
W1, b1 = self.params['W1'], self.params['b1']
W2, b2 = self.params['W2'], self.params['b2']
N, D = X.shape

# Compute the forward pass 
scores = None
u1 = X.dot(W1)+b1
z1 = self.relu(u1)
scores = z1.dot(W2)+b2


### Compute loss
계산해서 얻은 output이 실제랑 얼마나 잘 맞는지(해당 모델이 얼마나 잘 학습되었는지 확인하기 위해 loss를 계산한다.<br/>
loss function으로는 대표적으로 svm loss(hinge loss)와 softmax function을 많이 이용한다.<br/>
여기서는 svm loss를 활용하였다. 그리고 loss에 regularization term을 더해줌으로써 overfitting을 방지한다.

In [ ]:
# Compute the loss
loss = None
        
prob = np.exp(scores)/np.sum(np.exp(scores),axis=1,keepdims=True)
        
loss = np.sum(-np.log(prob[range(N),y]))/N
loss += 0.5 * reg * (np.sum(W1**2)+np.sum(W2**2))

**Softmax loss function **

In [ ]:
def softmax_loss(W, X, y, reg):
    """
    Softmax loss function, vectorized version.
    Inputs and outputs are the same as softmax_loss_naive.
    """
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)
    num_train = X.shape[0]
    num_classes = W.shape[1]
    
    score=X.dot(W)
    stable_score=score-np.array(num_classes*[np.max(score,axis=1),]).T
    prob = np.exp(stable_score)/np.sum(np.exp(stable_score),axis=1,keepdims=True)
    
    loss=np.sum(-np.log(prob[np.arange(0,prob.shape[0]),y]))
    loss=loss/num_train + 0.5 * reg * np.sum(W*W)  
    
    prob[np.arange(0,prob.shape[0]),y] -= 1
    
    dW = prob.T.dot(X).T
    
    dW /= num_train
    dW += reg*W

    pass

    return loss, dW

** SVM Loss **

In [ ]:
def svm_loss(W, X, y, reg):
    """
    Structured SVM loss function, vectorized implementation.
    Inputs and outputs are the same as svm_loss_naive.
    """
    loss = 0.0
    dW = np.zeros(W.shape) # initialize the gradient as zero

    pass
  
    num_classes = W.shape[1]
    num_train = X.shape[0]
    
    scores = X.dot(W)

    correct=scores[np.arange(0,scores.shape[0]),y]
    #dW[:,y[i]]+=-2*X[i]
        
    correct_score=np.array([correct,]*num_classes).T
    
    margin=scores-correct_score+1
    loss=np.sum(margin*(margin>0)*(margin!=1))/num_train
    loss += 0.5 * reg * np.sum(W * W)
     
    pass

    sum_or_not=np.array((margin>0)*(margin!=1),dtype=float)
    sum_or_not[np.arange(0,scores.shape[0]),y]=-np.sum(sum_or_not,axis=1)
    
    dW=np.dot(X.T,sum_or_not)
    
    dW/=num_train
    dW+=reg*W
    
    pass

    return loss, dW

## Back Propagation
back propagation은 gradient descent optimization방법을 이용해서 weight를 update해나가는 과정을 의미한다 <br/>
**Gradient Descent**란 가설함수의 local minimum지점을 찾기 위해서 현재 지점의 gradient(미분값)의 음수값에 비례하게 이동해가면서 minimum을 찾아나가는 optimization algorithm이다.  
![sgd](picture/sgd.png)
따라서 각각의 parameter(W1, b1, W2, b2)에 대해서 weight를 update하기 위해서는 loss function에 대해서 각각의 parameter에 대한 편미분(partial derivative)값을 구해줌으로써 그 값에 비례하게 weight를 update한다.
$$ new_w = w - (learning\,rate)* \frac{\delta\,loss}{\delta \,w} $$

In [ ]:
# Backward pass: compute gradients
dscore = prob.copy()
dscore[np.arange(0,prob.shape[0]),y] -= 1
dscore = dscore/N
        
grads = {}
grads['W2'] = z1.T.dot(dscore) + reg*W2
grads['b2'] = np.sum(dscore,axis=0) + reg*b2
       
tmp = dscore.dot(W2.T)
tmp[z1<=0]=0
grads['W1'] = X.T.dot(tmp) +reg*W1
grads['b1'] = np.sum(tmp,axis=0) + reg*b1


이러한 과정을 여러 번(iteration) 반복해주면 weight가 training data를 잘 설명해 줄 수 있는 값으로 최적화되고 training과정이 끝나게 된다.

# 3. Code

## Load data

In [4]:
%matplotlib inline

import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tools import rmse_cal,mae_cal,cor_cal,mean_cal,frange,\
                    accuracy,precision,recall,aupr,f1_score,make_binary
from validation import classification_cv,regression_cv,external_val_reg,\
                        external_val_classif, test_preprocessing, \
                        draw_roc,cal_external_auc

In [27]:
enc = OneHotEncoder()

dataset=pd.read_table('data/breast_cancer_svc.tsv',sep='\t')

input_data=dataset.iloc[:,1:].transpose()
X_data=input_data.iloc[:,:-1].values
y_data=input_data.iloc[:,-1]
y_data=make_binary('normal','cancer',y_data)


X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.1, random_state=42)

y_train = enc.fit_transform(y_train.values.reshape(-1,1)).toarray()
y_test = enc.fit_transform(y_test.values.reshape(-1,1)).toarray()

In [127]:
y_test.shape

(79, 2)

## Tensorflow

In [16]:
import tensorflow as tf

In [123]:
# variable 정의
X_dim = X_train.shape[1]
input_n = X_train.shape[0]
hidden_1 = 200
hidden_2 = 100
output_size = 2
lr = 5e-04
max_epoch = 50
dropout_prob = 0.5

In [124]:
# create fully connected layer
def dense(input_layer,layer_size):
    
    init = tf.contrib.layers.xavier_initializer()
    regularizer = tf.contrib.layers.l2_regularizer(scale=0.1)
    
    # Batchnorm settings
    #training_phase = tf.placeholder(tf.bool, phase, name='training_phase')
    
    HiddenLayer = tf.layers.dense(inputs = input_layer, units = layer_size, 
                              activation=None, # Batchnorm comes before nonlinear activation
                              use_bias=False, # Note that no bias unit is used in batchnorm
                              kernel_initializer=init, kernel_regularizer = regularizer)
    
    HiddenLayer = tf.layers.batch_normalization(HiddenLayer)
    HiddenLayer = tf.nn.relu(HiddenLayer)
    
    return HiddenLayer


In [125]:
tf.reset_default_graph()


# input 형태 정의
X = tf.placeholder(tf.float32, [None, X_dim])
y = tf.placeholder(tf.float32,[None,2])

with tf.name_scope("DNN"):
    
    keep_proba = tf.placeholder(tf.float32, None, name='keep_proba')

    
    hidden_layer1 = dense(X,hidden_1)
    hidden_layer2 = dense(hidden_layer1, hidden_2)
    hidden_layer2 = tf.nn.dropout(hidden_layer2, keep_prob=dropout_prob)
    output_layer = dense(hidden_layer2, output_size)
     
with tf.name_scope("loss"):
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=output_layer,labels=y)
    cost = tf.reduce_mean(loss, name = 'loss')

with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate = lr)
    training = optimizer.minimize(loss,name='training')
    
with tf.name_scope("eval"):
    correct = tf.equal(tf.argmax(y,1), tf.argmax(output_layer,1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32),name='eval')
    
    
init = tf.global_variables_initializer()


In [126]:
epoch_count = 0
best_accu_valid = 0
with tf.Session() as sess:
    init.run()
    while epoch_count < max_epoch :
        
        sess.run(training,feed_dict={X:X_train,y:y_train})
        accu_train = accuracy.eval(feed_dict={X:X_train,y:y_train})
        accu_valid = accuracy.eval(feed_dict={X:X_test,y:y_test})
        
        if accu_valid > best_accu_valid:
            best_accu_valid = accu_valid
            
        epoch_count+=1
        
        print('Epoch : ',epoch_count, '| Training Accuracy:',accu_train,'  Validation Accuracy:',accu_valid)

        

Epoch :  1 | Training Accuracy: 0.866009   Validation Accuracy: 0.873418
Epoch :  2 | Training Accuracy: 0.875882   Validation Accuracy: 0.911392
Epoch :  3 | Training Accuracy: 0.874471   Validation Accuracy: 0.911392
Epoch :  4 | Training Accuracy: 0.875882   Validation Accuracy: 0.911392
Epoch :  5 | Training Accuracy: 0.874471   Validation Accuracy: 0.911392
Epoch :  6 | Training Accuracy: 0.875882   Validation Accuracy: 0.911392
Epoch :  7 | Training Accuracy: 0.875882   Validation Accuracy: 0.911392
Epoch :  8 | Training Accuracy: 0.875882   Validation Accuracy: 0.911392
Epoch :  9 | Training Accuracy: 0.875882   Validation Accuracy: 0.911392
Epoch :  10 | Training Accuracy: 0.875882   Validation Accuracy: 0.911392
Epoch :  11 | Training Accuracy: 0.875881   Validation Accuracy: 0.911392
Epoch :  12 | Training Accuracy: 0.875882   Validation Accuracy: 0.911392
Epoch :  13 | Training Accuracy: 0.875882   Validation Accuracy: 0.911392
Epoch :  14 | Training Accuracy: 0.875882   Val

## Keras

In [134]:
import numpy as np
import pandas as pd
import os

from keras.models import Sequential
from keras.layers import Activation, Dropout, Dense
from keras import optimizers
from keras import applications
from keras.models import Model
from keras.regularizers import l2

Using TensorFlow backend.


In [148]:
hidden_1 = 128
hidden_2 = 64
dropout_rate = 0.5
batch_size = 25

epochs = 20
train_samples = X_train.shape[0]
validation_samples = X_test.shape[0]
x_dim = X_train.shape[1]

In [130]:
X_train.shape

(709, 294)

In [152]:
model = Sequential()

model.add(Dense(hidden_1,activation = 'relu',input_dim = x_dim))
model.add(Dense(hidden_2, activation = 'relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(2, activation = 'sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [153]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test,y_test), shuffle=True)

Train on 709 samples, validate on 79 samples
Epoch 1/20
709/709 [==============================] - 1s 1ms/step - loss: 0.3610 - acc: 0.8717 - val_loss: 0.2391 - val_acc: 0.9114
Epoch 2/20
709/709 [==============================] - 0s 617us/step - loss: 0.2789 - acc: 0.8942 - val_loss: 0.2002 - val_acc: 0.9557
Epoch 3/20
709/709 [==============================] - 0s 618us/step - loss: 0.2202 - acc: 0.9281 - val_loss: 0.1987 - val_acc: 0.9620
Epoch 4/20
709/709 [==============================] - 0s 608us/step - loss: 0.1760 - acc: 0.9337 - val_loss: 0.0994 - val_acc: 0.9747
Epoch 5/20
709/709 [==============================] - 0s 617us/step - loss: 0.1116 - acc: 0.9633 - val_loss: 0.0663 - val_acc: 0.9747
Epoch 6/20
709/709 [==============================] - 0s 606us/step - loss: 0.0737 - acc: 0.9810 - val_loss: 0.0388 - val_acc: 1.0000
Epoch 7/20
709/709 [==============================] - 0s 607us/step - loss: 0.0562 - acc: 0.9838 - val_loss: 0.0597 - val_acc: 0.9747
Epoch 8/20
709/709 

In [ ]:
# should install h5py

model.save_weights('log/dnn_weight1.h5')